In [1]:
%run basics
%matplotlib
import pysolar
import pytz

Using matplotlib backend: Qt4Agg


In [2]:
def solar_zenith_im(ldt,latitude,longitude,time_zone):
    # get a pytz object
    tz = pytz.timezone(time_zone)
    # use the same "current" date as Ian
    cdt = datetime.datetime(2013,6,1,0,0,0)
    # get the offset to UTC time as a timedelta object
    utcoffset = tz.utcoffset(cdt)
    # get the offset as hours
    GMT_zone = utcoffset.seconds/float(3600)
    # get the day of the year from local time
    DOY = numpy.array([dt.timetuple().tm_yday for dt in ldt])
    # For each day calculate equation of time correction, solar noon and declination
    array_EqofTime=0.17*numpy.sin(4*numpy.pi*(DOY-80)/373)-0.129*numpy.sin(2*numpy.pi*(DOY-8)/355) # DiLaura (1984)
    array_solar_noon=12+(GMT_zone*15.0-longitude)/360*24-array_EqofTime # Me
    array_decl=numpy.radians(23.4)*numpy.sin((DOY+284)/365.0*2*numpy.pi) # Oke (1987)
    # Create an hour angle array for each minute of day and each day of year
    array_h=numpy.tile(numpy.linspace(0,1439.0/1440*24,num=1440),(len(DOY),1))
    array_h=abs(numpy.radians((array_solar_noon.reshape(len(DOY),1)-array_h)*15))
    # Duplicate declination array for each time of day
    array_decl=numpy.tile(array_decl,(1440,1)).T
    # Calculate zenith angles
    array_z=numpy.arccos(numpy.sin(numpy.radians(latitude))*numpy.sin(array_decl)+numpy.cos(numpy.radians(latitude))*numpy.cos(array_decl)*numpy.cos(array_h))
    return array_z,array_h

In [3]:
def solar_zenith_pri(ldt,latitude,longitude,time_zone):
    # get a pytz object
    tz = pytz.timezone(time_zone)
    # use the same "current" date as Ian
    cdt = datetime.datetime(2013,6,1,0,0,0)
    # get the offset to UTC time as a timedelta object
    utcoffset = tz.utcoffset(cdt)
    # get the offset as hours
    GMT_zone = utcoffset.seconds/float(3600)
    # get the day of the year from local time
    DOY = numpy.array([dt.timetuple().tm_yday for dt in ldt])
    # For each day calculate equation of time correction, solar noon and declination
    eqn_of_time = 0.17*numpy.sin(4*numpy.pi*(DOY-80)/373)-0.129*numpy.sin(2*numpy.pi*(DOY-8)/355)
    solar_noon = 12+(GMT_zone*15.0-site_longitude)/360*24-eqn_of_time
    decln = numpy.radians(23.4)*numpy.sin((DOY+284)/365.0*2*numpy.pi)
    Hdh = numpy.array([dt.hour+dt.minute/float(60) for dt in ldt])
    Hdh = abs(numpy.radians((solar_noon-Hdh)*15))
    solar_zenith = numpy.arccos(numpy.sin(numpy.radians(site_latitude))
                                *numpy.sin(decln)
                                +numpy.cos(numpy.radians(site_latitude))
                                *numpy.cos(decln)*numpy.cos(Hdh))
    return solar_zenith,Hdh

In [4]:
def solar_altitude_from_zenith(solar_zenith):
    return numpy.pi/2-solar_zenith

In [5]:
site_name = "Whroo"
nc_name = "../Sites/Whroo/Data/Processed/2014/Whroo_2014_L3.nc"

In [6]:
ds = qcio.nc_read_series(nc_name)
site_timezone = ds.globalattributes["time_zone"]
site_latitude = float(ds.globalattributes["latitude"])
site_longitude = float(ds.globalattributes["longitude"])
ldt = ds.series["DateTime"]["Data"]

In [7]:
tz = pytz.timezone(site_timezone)
cdt = datetime.datetime(2013,6,1,0,0,0)
utcoffset = tz.utcoffset(cdt)
GMT_zone = utcoffset.seconds/float(3600)
DOY = numpy.array([dt.timetuple().tm_yday for dt in ldt])

In [9]:
sz_pri,Hdh = solar_zenith_pri(ldt,site_latitude,site_longitude,site_timezone)
sz_im,array_h = solar_zenith_im(ldt,site_latitude,site_longitude,site_timezone)

In [11]:
fig=plt.figure()
plt.plot(Hdh[0:47],sz_pri[0:47])
plt.show()

In [12]:
fig=plt.figure()
plt.plot(array_h[0,:],sz_im[0,:])
plt.show()

In [ ]:
sa_pri = solar_altitude_from_zenith(sz_pri)
sa_im = solar_altitude_from_zenith(sz_im)